In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,24617
2,Huelva,Confirmados PDIA 14 días,293
3,Huelva,Tasa PDIA 14 días,"57,09608901533605"
4,Huelva,Confirmados PDIA 7 días,132
5,Huelva,Total Confirmados,24837
6,Huelva,Curados,16619
7,Huelva,Fallecidos,305


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  24617.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6558.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 4795 personas en los últimos 7 días 

Un positivo PCR cada 1998 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,24617.0,132.0,293.0,513170.0,25.722470,57.096089,25.0
Huelva-Costa,15018.0,82.0,179.0,289548.0,28.320002,61.820493,17.0
Condado-Campiña,6888.0,48.0,105.0,156231.0,30.723736,67.208173,8.0
Puebla de Guzmán,92.0,10.0,12.0,3092.0,323.415265,388.098318,5.0
Huelva (capital),6558.0,30.0,72.0,143837.0,20.856942,50.056661,4.0
Almonte,867.0,17.0,31.0,24507.0,69.367936,126.494471,3.0
Cartaya,949.0,13.0,29.0,20083.0,64.731365,144.400737,3.0
Ayamonte,1196.0,10.0,11.0,21104.0,47.384382,52.122820,2.0
San Bartolomé de la Torre,195.0,3.0,4.0,3761.0,79.766020,106.354693,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Puebla de Guzmán,92.0,10.0,12.0,3092.0,323.415265,388.098318,5.0
Lucena del Puerto,154.0,7.0,8.0,3261.0,214.658080,245.323520,1.0
Cartaya,949.0,13.0,29.0,20083.0,64.731365,144.400737,3.0
Villablanca,176.0,2.0,4.0,2885.0,69.324090,138.648180,0.0
Palma del Condado (La),977.0,6.0,14.0,10801.0,55.550412,129.617628,0.0
Almonte,867.0,17.0,31.0,24507.0,69.367936,126.494471,3.0
San Bartolomé de la Torre,195.0,3.0,4.0,3761.0,79.766020,106.354693,2.0
Moguer,829.0,9.0,19.0,21867.0,41.157909,86.888919,0.0
Lepe,1994.0,7.0,24.0,27880.0,25.107604,86.083214,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Sierra de Huelva-Andévalo Central,2298.0,1.0,5.0,67391.0,1.483878,7.419388,0.0,0.200000
Punta Umbría,657.0,1.0,4.0,15355.0,6.512537,26.050147,0.0,0.250000
Palos de la Frontera,450.0,1.0,4.0,11742.0,8.516437,34.065747,1.0,0.250000
Bollullos Par del Condado,611.0,3.0,12.0,14387.0,20.852158,83.408633,1.0,0.250000
Lepe,1994.0,7.0,24.0,27880.0,25.107604,86.083214,0.0,0.291667
Rociana del Condado,310.0,1.0,3.0,7939.0,12.596045,37.788135,0.0,0.333333
Aljaraque,925.0,4.0,10.0,21474.0,18.627177,46.567943,1.0,0.400000
Huelva (capital),6558.0,30.0,72.0,143837.0,20.856942,50.056661,4.0,0.416667
Palma del Condado (La),977.0,6.0,14.0,10801.0,55.550412,129.617628,0.0,0.428571
